# Assembler in Python
    The assembler_interpreter function is a Python-based simulator of an assembly-like language. 
    Some programs are shown as example. 
    Task from 'Codewars', https://www.codewars.com/kata/58e61f3d8ff24f774400002c.

## Assembler

In [1]:
def assembler_interpreter(program):
    
    #Convert input to a list of the lines 
    program = program.split("\n")[1:-1]
    
    d, i = {"pointer":[]}, 0
    
    #Find functions
    for i in range(len(program)): 
        if program[i] != "":
            if program[i][-1] == ":": d[program[i][:-1]] = i #PROBLEM: Does also recognize comments ending on ":"
        
    #Commands
    i = 0
    while i < len(program):
        cmd, r, v = (program[i] + ' 0' + " 0" + " 0").replace(",", "").split()[:3]
        
        #Simple commands
        if cmd == 'inc': d[r] += 1
        elif cmd == 'dec': d[r] -= 1        
        elif cmd == 'mov': d[r] = d[v] if v in d else int(v)
        elif cmd == 'jnz' and (d[r] if r in d else int(r)): i += int(v) - 1
        elif cmd == "add": d[r] += d[v] if v in d else int(v)
        elif cmd == "sub": d[r] -= d[v] if v in d else int(v)
        elif cmd == "mul": d[r] *= d[v] if v in d else int(v)
        elif cmd == "div": d[r] //= d[v] if v in d else int(v)
        
        #Advanced commands
        elif cmd == ";": pass
        elif cmd == "msg":
            msg = ""
            statement_list = []
            string = program[i].replace("msg", "").split(";")[0]

            quote_flag = False
            for k in string:
                if k == "'": 
                    quote_flag = not quote_flag            #Switch quote_flag on " ' "
                    if quote_flag: word = ""               #Form a new word
                    else: statement_list.append(word)      #append the word when quote is finished

                elif quote_flag:
                    word += k
                elif k not in [" ", ","]:                  # Don't add "," and " " to the registries you seek!
                            statement_list.append(k)       # Can't handle 2 character registers yet

            for m in statement_list:
                msg += str(d[m]) if m in d else m
            d["output"] = msg
        elif cmd == "end": return d["output"]
            
        #Labels, Functions and Jumps
        elif cmd == "call": 
            d["pointer"].append(i)
            i = d[r]
        elif cmd == "ret": 
            i = d["pointer"][-1]                                     #return to last pointer
            d["pointer"] = d["pointer"][:-1]
        elif cmd == "jmp": i = d[r] 
        elif cmd == "cmp": d["cmp"] = (d[r] if r in d else int(r)) - (d[v] if v in d else int(v))                 
        elif cmd == "jne": i = d[r] if d["cmp"] != 0 else i
        elif cmd == "je": i = d[r] if d["cmp"] == 0 else i                     
        elif cmd == "jge": i = d[r] if d["cmp"] >= 0 else i
        elif cmd == "jg": i = d[r] if d["cmp"] > 0 else i                       
        elif cmd == "jle": i = d[r] if d["cmp"] <= 0 else i                        
        elif cmd == "jl": i = d[r] if d["cmp"] < 0 else i                        
                                  
        i += 1

    return -1

## Program Examples

In [2]:
program_factorial = '''
mov   a, 5
mov   b, a
mov   c, a
call  proc_fact
call  print
end

proc_fact:
    dec   b
    mul   c, b
    cmp   b, 1
    jne   proc_fact
    ret

print:
    msg   a, '! = ', c ; output text
    ret
'''

print(assembler_interpreter(program_factorial))

5! = 120


In [3]:
program_fibonacci = '''
mov   a, 8            ; value
mov   b, 0            ; next
mov   c, 0            ; counter
mov   d, 0            ; first
mov   e, 1            ; second
call  proc_fib
call  print
end

proc_fib:
    cmp   c, 2
    jl    func_0
    mov   b, d
    add   b, e
    mov   d, e
    mov   e, b
    inc   c
    cmp   c, a
    jle   proc_fib
    ret

func_0:
    mov   b, c
    inc   c
    jmp   proc_fib

print:
    msg   'Term ', a, ' of Fibonacci series is: ', b        ; output text
    ret
'''

print(assembler_interpreter(program_fibonacci))

Term 8 of Fibonacci series is: 21


In [4]:
program_mod = '''
mov   a, 11           ; value1
mov   b, 3            ; value2
call  mod_func
msg   'mod(', a, ', ', b, ') = ', d        ; output
end

; Mod function
mod_func:
    mov   c, a        ; temp1
    div   c, b
    mul   c, b
    mov   d, a        ; temp2
    sub   d, c
    ret
'''

print(assembler_interpreter(program_mod))

mod(11, 3) = 2


In [5]:
program_gcd = '''
mov   a, 81         ; value1
mov   b, 153        ; value2
call  init
call  proc_gcd
call  print
end

proc_gcd:
    cmp   c, d
    jne   loop
    ret

loop:
    cmp   c, d
    jg    a_bigger
    jmp   b_bigger

a_bigger:
    sub   c, d
    jmp   proc_gcd

b_bigger:
    sub   d, c
    jmp   proc_gcd

init:
    cmp   a, 0
    jl    a_abs
    cmp   b, 0
    jl    b_abs
    mov   c, a            ; temp1
    mov   d, b            ; temp2
    ret

a_abs:
    mul   a, -1
    jmp   init

b_abs:
    mul   b, -1
    jmp   init

print:
    msg   'gcd(', a, ', ', b, ') = ', c
    ret
'''

print(assembler_interpreter(program_gcd))

gcd(81, 153) = 9


In [6]:
program_power = '''
mov   a, 2            ; value1
mov   b, 10           ; value2
mov   c, a            ; temp1
mov   d, b            ; temp2
call  proc_func
call  print
end

proc_func:
    cmp   d, 1
    je    continue
    mul   c, a
    dec   d
    call  proc_func

continue:
    ret

print:
    msg a, '^', b, ' = ', c
    ret
'''

print(assembler_interpreter(program_power))

2^10 = 1024
